In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt
import matplotlib.image as imgs
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchsummary import summary as summary
from torchvision import transforms
from PIL import Image
import sys
import pandas as pd



class CustomDataset(Dataset): 
    def __init__(self, x_data, y_data, transform=None):
        # self.x_train = np.array(x_data)
        self.y_train = np.array(y_data)
        self.x_data = x_data
        self.y_data = y_data 
        self.transform = transform
        self.target_transform = ToTensor()
        self.idx = 0
    def __len__(self):
            return len(self.x_data)
    def __getitem__(self, idx):
            if self.transform:
                x = self.transform(self.x_data[idx])
                self.idx = idx 
            if self.target_transform:
                y = torch.from_numpy(np.array(self.y_train[idx])).float()
            return x, y
    def __idx__(self):
        return self.x_data[self.idx]
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        output_channel = 512
        input_channel = 3
        self.output_channel = [int(output_channel / 8), int(output_channel / 4),
        int(output_channel / 2), output_channel]  # [64, 128, 256, 512]
        
        self.ConvNet = nn.Sequential(
            nn.Conv2d(in_channels=input_channel, out_channels=self.output_channel[0], kernel_size=3, stride=1, padding=1), nn.ReLU(True),
            nn.MaxPool2d(2, 2),  # 64x16x50
            nn.Conv2d(in_channels=self.output_channel[0], out_channels=self.output_channel[1], kernel_size=3, stride=1, padding=1), nn.ReLU(True),
            nn.MaxPool2d(2, 2),  # 128x8x25
            nn.Conv2d(in_channels=self.output_channel[1], out_channels=self.output_channel[2],kernel_size=3, stride=1, padding=1), nn.ReLU(True),  # 256x8x25
            nn.Conv2d(in_channels=self.output_channel[2], out_channels=self.output_channel[2], kernel_size=3, stride=1, padding=1), nn.ReLU(True),
            nn.MaxPool2d((2, 1), (2, 1)),  # 256x4x25
            nn.Conv2d(in_channels=self.output_channel[2], out_channels=self.output_channel[3], kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.output_channel[3]), nn.ReLU(True),  # 512x4x25
            nn.Conv2d(in_channels=self.output_channel[3], out_channels=self.output_channel[3], kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.output_channel[3]), nn.ReLU(True),
            nn.MaxPool2d((2, 1), (2, 1)),  # 512x2x25
            nn.Conv2d(in_channels=self.output_channel[3], out_channels=self.output_channel[3], kernel_size=3, stride=1, padding=1), nn.ReLU(True),
            nn.MaxPool2d((2, 1), (2, 1)))  # 512x1x24
            # nn.AdaptiveAvgPool2d((1, 1)))
        self.fc_layer = nn.Sequential(
        	# [100,64*3*3] -> [100,100]
            nn.Linear(65536,4096),                                              
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,1000),  
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1000,2),                             
        )     
    def forward(self, x):
        out = self.ConvNet(x)
        out = torch.flatten(out, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        out = self.fc_layer(out)
        return out

class NeuralNetwork2(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        output_channel = 512
        input_channel = 3
        self.output_channel = [int(output_channel / 8), int(output_channel / 4),
        int(output_channel / 2), output_channel]  # [64, 128, 256, 512]
        
        self.ConvNet = nn.Sequential(
            nn.Conv2d(in_channels=input_channel, out_channels=self.output_channel[0], kernel_size=3, stride=1, padding=1), nn.ReLU(True),
            nn.MaxPool2d(2, 2),  # 64x16x50
            nn.Conv2d(in_channels=self.output_channel[0], out_channels=self.output_channel[1], kernel_size=3, stride=1, padding=1), nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(in_channels=self.output_channel[1], out_channels=self.output_channel[2],kernel_size=3, stride=1, padding=1), nn.ReLU(True),  # 256x8x25
            nn.Conv2d(in_channels=self.output_channel[2], out_channels=self.output_channel[2], kernel_size=3, stride=1, padding=1), nn.ReLU(True),
            nn.MaxPool2d((2, 1), (2, 1)),  # 256x4x25
            nn.Conv2d(in_channels=self.output_channel[2], out_channels=self.output_channel[2], kernel_size=3, stride=1, padding=1), nn.ReLU(True),
            nn.BatchNorm2d(self.output_channel[2]), nn.ReLU(True),  # 512x4x25
            nn.Conv2d(in_channels=self.output_channel[2], out_channels=self.output_channel[2], kernel_size=3, stride=1, padding=1), nn.ReLU(True),
            nn.BatchNorm2d(self.output_channel[2]), nn.ReLU(True),  # 512x4x25
            nn.Conv2d(in_channels=self.output_channel[2], out_channels=self.output_channel[2], kernel_size=3, stride=1, padding=1), nn.ReLU(True),
            nn.MaxPool2d((2, 1), (2, 1)),  # 256x4x25
            nn.Conv2d(in_channels=self.output_channel[2], out_channels=self.output_channel[3], kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.output_channel[3]), nn.ReLU(True),  # 512x4x25
            nn.Conv2d(in_channels=self.output_channel[3], out_channels=self.output_channel[3], kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.output_channel[3]), nn.ReLU(True),
            nn.MaxPool2d((2, 1), (2, 1)), 
            nn.Conv2d(in_channels=self.output_channel[3], out_channels=self.output_channel[3], kernel_size=3, stride=1, padding=1), nn.ReLU(True),
            # nn.MaxPool2d((2, 1), (2, 1)))  # 512x1x24
            nn.AdaptiveAvgPool2d((1, 1)))
        self.fc_layer = nn.Sequential(
        	# [100,64*3*3] -> [100,100]
            # nn.Linear(28672,4096),                                              
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Linear(4096,1000),  
            # nn.ReLU(),
            # nn.Dropout(0.5),
            nn.Linear(512,2),                             
        )     
    def forward(self, x):
        out = self.ConvNet(x)
        out = torch.flatten(out, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        out = self.fc_layer(out)
        # nn.Linear(1024,2)
        return out
        
    
def file_recursively(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if((os.path.join(root,file).find(".jpg")) != -1):
                list_root.append(os.path.join(root ,file))
             
def train_test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.type(torch.LongTensor)
            y = y.to(device)
            output = model.forward(x)
            loss = loss_fn(output,y)
            test_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(y.view_as(pred)).sum().item()
            
    test_loss /= size
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(dataloader.dataset), 100. * correct / len(dataloader.dataset)))
    f.write('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(dataloader.dataset), 100. * correct / len(dataloader.dataset)))
      
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.type(torch.LongTensor)
            y = y.to(device)
            output = model.forward(x)
            loss = loss_fn(output,y)
            test_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(y.view_as(pred)).sum().item()
            
    test_loss /= size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(dataloader.dataset), 100. * correct / len(dataloader.dataset)))
    f.write('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(dataloader.dataset), 100. * correct / len(dataloader.dataset)))
    return test_loss
if __name__ == '__main__':
                     
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )

    print(f"Using {device} device")   

    list_root = []
    dic = {}
    list_root_label = []

    file_recursively('E:/동현_개인폴더/대회/USG제조혁신대회2차/train_img5')
    label = pd.read_csv('E:/동현_개인폴더/대회/USG제조혁신대회2차/train.csv')
    
    img_end = []
    img_end2 = []
    
    for i in list_root:
        # img = imgs.imread(i)
        img = Image.open(i).convert("RGB")
        # img = cv2.resize(img, dsize=(640,640), interpolation=cv2.INTER_LINEAR)
        min_value = np.min(img)
        max_value = np.max(img)
        output = (img - min_value) / (max_value - min_value)
        img_end.append(output)
        img_end2.append(img)

    x_train, x_valid, y_train, y_valid = train_test_split(img_end2, label['label'], test_size= 0.2, random_state= 33 ,shuffle=True)
            
            
    #배치 사이즈 조정
    batch_size = 2


    transforms_train = transforms.Compose([ transforms.Resize((128, 128)),
                                        transforms.ToTensor(),
                                        # transforms.RandomRotation(10.)
                                        ])

    transforms_test = transforms.Compose([transforms.Resize((128,128)),
                                        transforms.ToTensor()
                                        ])


    dataset_train = CustomDataset(x_train, y_train, transform=transforms_train)
    dataset_valid = CustomDataset(x_valid, y_valid, transform=transforms_test)


    dataloader_train = DataLoader(dataset_train, batch_size=batch_size, num_workers=0, shuffle=True)
    dataloader_valid = DataLoader(dataset_valid, batch_size=batch_size, num_workers=0, shuffle=True)
    model = NeuralNetwork().to(device)
    # optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 


    print(dataloader_train)

    #-------------------------학습률 등의 파라미터 조정------------------------------------#
    
    
    learning_rate = 1e-5
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_func = nn.CrossEntropyLoss()

    # dataloader_train = dataloader_train.squeeze(dim=0)
    nb_epochs = 100
    txtName = "modelTrainLog"
    count = 1
    best_loss = 10 ** 9 # 매우 큰 값으로 초기값 가정
    patience_limit = 10 # 몇 번의 epoch까지 지켜볼지를 결정
    patience_check = 0 # 현재 몇 epoch 연속으로 loss 개선이 안되는지를 기록
    
    
    #------------------모델 학습 기록 생성-----------------------------------------------#
    
    if os.path.isfile("./model/{}.txt".format(txtName)) != True:
        f = open("./model/{}.txt".format(txtName), "w")
    else:
        while(True):
            if os.path.isfile("./model/{} ({}).txt".format(txtName, count)) != True:
                f = open("./model/{} ({}).txt".format(txtName, count), "w")
                break
            else:
                count += 1
                
                
                
    summary(model, (3,128, 128))
    f.write(f"batchsize: {batch_size:>d} epochs: {nb_epochs:>d}\n\n")




    #------------------모델 학습-----------------------------------------------#

    for epoch in range(nb_epochs + 1):
        for batch, (x, y) in enumerate(dataloader_train):
            x_train = x.to(device)
            y = y.type(torch.LongTensor)
            y_train = y.to(device)
            # H(x) 계산
            optimizer.zero_grad()

            pred = model.forward(x_train)
            
            loss = loss_func(pred, y_train)
         
            # 역전파
            loss.backward()
            optimizer.step()
            if batch % 1000 == 0:
                loss, current = loss.item(), (batch + 1) * len(x_train)
                # print(f"[epoch: {epoch:>5d}] loss: {loss:>7f}")
                print(f"[epoch: {epoch:>d}] loss: {loss:>7f}  [{current:>5d}/{len(dataset_train.x_data):>5d}]")
                f.write(f"[epoch: {epoch:>d}] loss: {loss:>7f}  [{current:>5d}/{len(dataset_train.x_data):>5d}]\n")
        train_test(dataloader_train, model, loss_func)   
        val_loss = test(dataloader_valid, model, loss_func)
        model.eval()
            
        ### early stopping 여부를 체크하는 부분 ###
        if val_loss > best_loss: # loss가 개선되지 않은 경우
            patience_check += 1
            if patience_check >= 5:
                learning_rate = learning_rate / 2
            if patience_check >= patience_limit: # early stopping 조건 만족 시 조기 종료
                break
        else:
            best_loss = val_loss
            patience_check = 0
        PATH = './model/vgg_net_check_point.pth' #체크포인트
        torch.save(model, PATH)
        
    PATH = './model/vgg_net.pth'
    torch.save(model, PATH)
    PATH = './model/vgg_best.pth'
    torch.save(model.state_dict(), PATH)
        
    model.eval()


    #최종 평가 결과 및 miscls 파악
    with torch.no_grad():
        correct = 0
        total = 0
        count = 0
        for x,y in dataloader_valid:
            x = x.to(device)
            y = y.type(torch.LongTensor)
            y = y.to(device)
            outputs = model.forward(x)
            _, predicted = torch.max(outputs.data, 1)
            total += len(y)
            correct += (predicted == y).sum().item()
            for i in range(0, len(y)):
                if(predicted[i].item() != y[i]):
                    plt.axis('off')
                    tf = transforms.ToPILImage()
                    img = tf(x[i])
                    plt.imshow(img)
                    plt.savefig('./mis2/{}_{}_{}.jpg'.format(count,y[i], predicted[i].item()),format='jpeg',bbox_inches="tight", pad_inches = 0)
                count += 1
        print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total))
        f.write('Test Accuracy of the model on the {} test images: {} %\n'.format(total, 100 * correct / total))
    f.close()
    
    

Using cuda device
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           1,792
              ReLU-2         [-1, 64, 128, 128]               0
         MaxPool2d-3           [-1, 64, 64, 64]               0
            Conv2d-4          [-1, 128, 64, 64]          73,856
              ReLU-5          [-1, 128, 64, 64]               0
         MaxPool2d-6          [-1, 128, 32, 32]               0
            Conv2d-7          [-1, 256, 32, 32]         295,168
              ReLU-8          [-1, 256, 32, 32]               0
            Conv2d-9          [-1, 256, 32, 32]         590,080
             ReLU-10          [-1, 256, 32, 32]               0
        MaxPool2d-11          [-1, 256, 16, 32]               0
           Conv2d-12          [-1, 512, 16, 32]       1,179,648
      BatchNorm2d-13          [-1, 512, 16, 32]           1,024
             ReLU-14 

KeyboardInterrupt: 